# WebSci19 experiments



In [54]:
%load_ext autoreload
%autoreload 2
import plotly.offline as py
import helper
helper.bodge_path()

from occ.model import follow1, follow2, modulated_internal
from occ.model import UnitModel, u, n1, n2, Unit, marking
from occ.sim import run, SystemModel, SimArgs
from occ.vis import iplot_causal_graph
from helper import two_community_network, generate_place_pair_with_random_markings, generate_place_pair_with_marking

import logging
logging.getLogger().setLevel(logging.WARNING)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Runs will be written to:

In [41]:
print(occ.sim.BASEDIR)

/Users/walton/Documents/DPhil/code/occmodeller/runs


In [42]:
sim_args = SimArgs(start=0, stop=20, step=.01)

## Diffusion (3.4.1)

In [43]:
g = two_community_network()

indexes = list(range(g.number_of_nodes()))
launch_node = 0
indexes.remove(launch_node)

a = Unit.place('a', marking(indexes))
b = Unit.place('b', marking([launch_node]))
m = UnitModel(name="diffusion", colors=[Unit], variables=[u, n1, n2],places=[a, b])
m.add_transitions_from([
    follow1(a, b, use_read_arc=False)
])

In [44]:
sim_result = run(SystemModel(m, g, 'two community'), sim_args)

In [45]:
iplot_causal_graph(sim_result)

## Multiphase diffusion (3.4.2)

In [20]:
indexes = list(range(14))
for i in [5, 4, 3]:
    indexes.remove(i)

a = Unit.place('a', marking(indexes))
b = Unit.place('b', marking([4, 3]))
c = Unit.place('c', marking([5]))
m = UnitModel(name="multiphase_diffusion", colors=[Unit], variables=[u, n1, n2], places=[a, b])
m.add_transitions_from([
    follow1(a, b),
    follow1(b, c)
])
medium_graph = nx.newman_watts_strogatz_graph(14, 4, 0.2, seed=5)

In [46]:
iplot_causal_graph(run(SystemModel(m, medium_graph, 'one community'), sim_args))

## Diffusion with echo (3.4.3)

The first model requres two neighbours to transition

In [47]:
g = two_community_network()
indexes = list(range(g.number_of_nodes()))
init1 = 3
init2 = 4

indexes.remove(init1)
indexes.remove(init2)

a = Unit.place('a', marking(indexes))
b = Unit.place('b', marking([init1, init2]))
m = UnitModel(name="two_neighbour_diffusion", colors=[Unit], variables=[u, n1, n2],
              places=[a, b])
m.add_transitions_from([
    follow2(a, b, 1),
])


In [48]:
iplot_causal_graph(run(SystemModel(m, g, 'two community'), sim_args))

We now add a second transition:

In [36]:
m.add_transition(follow1(a, b, .5))

In [37]:
iplot_causal_graph(run(SystemModel(m, g, 'two community'), sim_args))

INFO:root:Calling: '['~/bin/spike exe -f input/conf.spc']' from cwd: /Users/walton/Documents/DPhil/code/occmodeller/runs/32/spike
INFO:root:Call to Spike complete


## Single issue consensus formation (3.4.4)

In [51]:
g = two_community_network()

a, A = generate_place_pair_with_random_markings('a', 'A', g.number_of_nodes())

m = UnitModel(name="single_issue_consensus", colors=[Unit], variables=[u, n1, n2],
              places=[a, A])
m.add_transitions_from([
    follow1(a, A, .5),
    follow1(A, a, .5),
    follow2(a, A, 1),
    follow2(A, a, 1),
])


In [52]:
iplot_causal_graph(run(SystemModel(m, g, 'two community'), sim_args))

## Issue conflation (3.4.5)

In [55]:
g = two_community_network()
n = g.number_of_nodes()

def triangle_coords_around_zero(edge_width):
    a = (-.5 * edge_width, -edge_width * .866 / 2)
    b = (.5 * edge_width, -edge_width * .866 / 2)
    c = (0 * edge_width, edge_width * 0.866 / 2)
    return a, b, c

a_off, b_off, e_off = triangle_coords_around_zero(.05)
a, A = generate_place_pair_with_random_markings('a', 'A', n, pos_offset=a_off)
b, B = generate_place_pair_with_random_markings('b', 'B', n, pos_offset=b_off)
e, E = generate_place_pair_with_marking('e', 'E', n, list(range(int(n / 2))), pos_offset=e_off)

m = UnitModel(name="conflation_through_emotion", colors=[Unit], variables=[u, n1, n2],
              places=[a, A, b, B, e, E])

r_f1 = 1
r_f2 = 2
r_e_f1 = r_f1
r_e_f2 = r_f2
r_ab_to_e = 1

m.add_transitions_from([
    follow1(a, A, r_f1, enabled_by_local=E),
    follow1(A, a, r_f1, enabled_by_local=e),

    follow1(b, B, r_f1, enabled_by_local=E),
    follow1(B, b, r_f1, enabled_by_local=e),

    follow1(e, E, r_e_f1),
    follow1(E, e, r_e_f1),

    modulated_internal(A, a, e, rate=r_ab_to_e),
    modulated_internal(B, b, e, rate=r_ab_to_e),
    modulated_internal(a, A, E, rate=r_ab_to_e),
    modulated_internal(b, B, E, rate=r_ab_to_e),
])


In [56]:
iplot_causal_graph(run(SystemModel(m, g, 'two community'), sim_args))

*** nan b 0.1
*** nan B 0.33
*** nan A 0.82
*** nan A 0.88
*** nan a 1.23
*** nan b 3.78
*** nan A 4.36
*** nan b 5.6
*** nan a 5.81
*** nan b 5.91
*** nan a 5.99
*** nan b 6.48
